## Combined and Write Training Data

In [ ]:
import os
import types
import pandas as pd

In [ ]:
# Set up data directory
CWD = os.getcwd()
cms_data_dir = os.path.join(CWD, 'CMSData')

In [ ]:
# Some years columns are capitalized and other years the columns are lowercase:
capitalization_dict = {
    '2012': str.upper,
    '2013': str.upper,
    '2014': str.lower,
    '2015': str.lower,
    '2016': str.upper,
    '2017': str.lower,
}

### 1. CMS Part B dataset

In [ ]:
# Set dtypes based on https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/...
#Medicare-Provider-Charge-Data/Physician-and-Other-Supplier2017
partB_dtypes = {
    'npi': 'int64',
    'nppes_provider_last_org_name': 'str',
    'nppes_provider_first_name': 'str',
    'nppes_provider_mi': 'str',
    'nppes_credentials': 'str',
    'nppes_provider_gender': 'str',
    'nppes_entity_code': 'str',
    'nppes_provider_street1': 'str',
    'nppes_provider_street2': 'str',
    'nppes_provider_city': 'str',
    'nppes_provider_zip': 'str',
    'nppes_provider_state': 'str',
    'nppes_provider_country': 'str',
    'provider_type': 'str',
    'medicare_participation_indicator': 'str',
    'place_of_service': 'str',
    'hcpcs_code': 'str',
    'hcpcs_description': 'str',
    'hcpcs_drug_indicator': 'str',
    'line_srvc_cnt': 'float64',
    'bene_unique_cnt': 'float64',    
    'bene_day_srvc_cnt': 'float64',
    'average_medicare_allowed_amt': 'float64',
    'average_submitted_chrg_amt': 'float64',
    'average_medicare_payment_amt': 'float64',
    'average_medicare_standard_amt': 'float64',
}

In [ ]:
# Get dfs for all years - TAKE A FEW MINUTES
years = ['2012','2013','2014','2015','2016','2017']
dfs   = []

for year in years:
    file = os.path.join(cms_data_dir, f'cms{year}.txt')
    dtypes = dict(zip(list(map(capitalization_dict[year], partB_dtypes.keys())), list(partB_dtypes.values()))) #get correct column capitalization and dtype
    df = pd.read_csv(file, delimiter='\t', dtype=dtypes)
    df.columns = map(str.lower, df.columns)  # make all variable names lowercase
    df['year'] = year #add Year column 
    dfs.append(df)

In [ ]:
# Concatenate
partB_df = pd.concat(dfs, axis=0, ignore_index=True, sort=False)
partB_df.shape

In [ ]:
# Drop missing NPI and HCPCS - "Medicare fraud detection using neural networks" (Johnson, Khoshgoftaar 2019)
partB_df = partB_df.dropna(subset = ['npi','hcpcs_code'])
partB_df.shape

In [ ]:
# Remove rows corresponding to drugs because LINE_SRVC_CNT for them is not a desirable count
partB_df = partB_df[(partB_df['hcpcs_drug_indicator'] == 'N')]
partB_df.shape

In [ ]:
# Keep variables based on "Medicare fraud detection using neural networks" (Johnson, Khoshgoftaar 2019)
partB_variables_to_keep = [
    'npi',
    'provider_type',
    'nppes_provider_city', # keep
    'nppes_provider_zip', # keep
    'nppes_provider_state', # keep
    'nppes_provider_country', # keep
    'hcpcs_code',  # not in paper but kept
    'hcpcs_description',  # not in paper but kept
    'hcpcs_drug_indicator',  # not in paper but kept
    'place_of_service',  # not in paper but kept
    'nppes_provider_gender',
    'line_srvc_cnt',
    'bene_unique_cnt',
    'bene_day_srvc_cnt',
    'average_submitted_chrg_amt',
    'average_medicare_payment_amt',
    'year' # need Year for labeling
]
partB_df = partB_df[partB_variables_to_keep]

In [ ]:
partB_df.head()

In [ ]:
partB_df.loc[partB_df['npi'] == 1003000142][['npi',
                                             'provider_type',
                                             'place_of_service',
                                             'line_srvc_cnt',
                                             'average_submitted_chrg_amt']][:5]

In [ ]:
partB_df['year'].value_counts()

In [ ]:
# Write all combined CMS to csv
#partB_df.to_csv('combined-partB-data-v2')

### 2. LEIE Dataset

In [ ]:
leie_data_dir = os.path.join(CWD, 'LEIEData')

In [ ]:
leie_dtypes = {
    'LASTNAME': 'str',
    'FIRSTNAME': 'str',
    'MIDNAME': 'str',
    'BUSNAME' : 'str',
    'GENERAL': 'str',
    'SPECIALTY': 'str',
    'UPIN': 'str',
    'NPI': 'int64',
    'DOB': 'str',
    'ADDRESS': 'str',
    'CITY': 'str',
    'STATE': 'str',
    'ZIP': 'str',
    'EXCLTYPE': 'str',
    'EXCLDATE': 'int64',
    'REINDATE': 'int64',
    'WAIVERDATE': 'int64',
    'WVRSTATE': 'str',
}

In [ ]:
#LEIE data is monthly between 01/2018 (1801) - 12/2019 (1912)
year_months = ['1801','1802','1803','1804','1805','1806','1807','1808','1809','1810','1811','1812',
            '1901','1902','1903','1904','1905','1906','1907','1908','1909','1910','1911','1912']
dfs = []

for year_month in year_months:
    file = os.path.join(leie_data_dir, f'leie{year_month}-excl.csv')
    df   = pd.read_csv(file, dtype=leie_dtypes)
    df.columns = map(str.lower, df.columns)
    dfs.append(df)

In [ ]:
# Concatenate
leie_df = pd.concat(dfs, axis=0, ignore_index=True, sort=False)
leie_df.shape

In [ ]:
leie_df.head()

In [ ]:
# Drop NPI = 0, which means missing - A LOT ARE MISSING, which is a problem for the data
leie_df = leie_df[leie_df['npi'] != 0]
leie_df.shape

In [ ]:
# Keep exclusions most related to Fraud
exclusions_to_keep = [
    '1128a1',
    '1128a2',
    '1128a3',
    '1128b4',
    '1128b7',
    '1128c3Gi',
    '1128c3gii',
]
leie_df = leie_df[leie_df['excltype'].isin(exclusions_to_keep)]
leie_df.shape

In [ ]:
leie_df['excltype'].value_counts()

In [ ]:
# Write all combined LEIE to csv
#partB_df.to_csv('combined-leie-data')

### 3. Combine/Label Data

In [ ]:
from datetime import datetime, timedelta
import numpy as np

In [ ]:
# Convert to datetime
leie_df['excldate'] = pd.to_datetime(leie_df['excldate'], format='%Y%m%d', errors ='ignore')

In [ ]:
# Round excl date to the nearest year Johnson & Khoshgoftaar (2019)
def round_to_year(dt=None):
    year = dt.year
    month = dt.month
    if month >= 6:
        year = year + 1
    return datetime(year=year,month=1,day=1)

leie_df['excl_year'] = leie_df.excldate.apply(lambda x: round_to_year(x))

In [ ]:
# Make exclusion dict 
# 1215053665 has 2 exclusions, so sort df to get latest year
excl_year_dict = dict([npi, year] for npi, year in zip(leie_df.sort_values(by='excl_year').npi, leie_df.sort_values(by='excl_year').excl_year))

In [ ]:
# Get label as 0 or 1
partB_df['excl_year'] = partB_df['npi'].map(excl_year_dict)
partB_df['excl_year'] = partB_df['excl_year'].fillna(datetime(year=1900,month=1,day=1)) # fill NaN, physicians without exclusion, with year 1900

partB_df['year'] = pd.to_datetime(partB_df['year'].astype(str), format='%Y', errors ='ignore')
partB_df['fraudulent'] = np.where(partB_df['year'] < partB_df['excl_year'], 1, 0) # compare year vs. exclusion year to get Fraudulent

### 4. Prep Visualization + Training Data

- Aggregate data following paper's method
- Write data for visualization (including state, city, zip)
- Normalize predictors
- One hot encoding
- Write data for training (excluding state, city, zip)

In [ ]:
new_variables_to_keep = [
    'year',
    'npi',
    'provider_type',
    'nppes_provider_city',   #only keep for visualization
    'nppes_provider_state',  #only keep for visualization
    'nppes_provider_country',#only keep for visualization
    'nppes_provider_gender',
    'line_srvc_cnt',
    'bene_unique_cnt',
    'bene_day_srvc_cnt',
    'average_submitted_chrg_amt',
    'average_medicare_payment_amt',
    'fraudulent'
]

In [ ]:
#group by
temp_df = partB_df[new_variables_to_keep]

#agg by npi - provider_type and get sum stats
agg_partB_df = temp_df.groupby(by=['year','npi','provider_type','nppes_provider_city',
                      'nppes_provider_state','nppes_provider_country','nppes_provider_gender']).agg(
                    {
                    'line_srvc_cnt':["mean","median","std", min,max,sum],
                    'bene_unique_cnt':["mean","median","std", min,max,sum],
                    'bene_day_srvc_cnt':["mean","median","std", min,max,sum],
                    'average_submitted_chrg_amt':["mean","median","std", min,max,sum],
                    'average_medicare_payment_amt':["mean","median","std", min,max,sum],
                    'fraudulent':["mean"],
                    }).reset_index()

agg_partB_df.columns = ["_".join(x) for x in agg_partB_df.columns.ravel()] #unravel to get rid of multi-index column names

In [97]:
agg_partB_df.head()

,year_,npi_,provider_type_,nppes_provider_city_,nppes_provider_state_,nppes_provider_country_,nppes_provider_gender_,line_srvc_cnt_mean,line_srvc_cnt_median,line_srvc_cnt_std,...,average_submitted_chrg_amt_min,average_submitted_chrg_amt_max,average_submitted_chrg_amt_sum,average_medicare_payment_amt_mean,average_medicare_payment_amt_median,average_medicare_payment_amt_std,average_medicare_payment_amt_min,average_medicare_payment_amt_max,average_medicare_payment_amt_sum,fraudulent_mean
0,2012-01-01,1003000126,Internal Medicine,CUMBERLAND,MD,US,M,174.857143,111.0,166.951518,...,58.000000,291.000000,1060.000000,82.218697,81.390000,41.942570,30.720721,158.870000,575.530877,0
1,2012-01-01,1003000134,Pathology,EVANSTON,IL,US,M,959.125000,223.0,2076.546889,...,39.000000,263.000000,1065.000000,26.053306,25.187934,18.836154,7.815385,64.015735,208.426446,0
2,2012-01-01,1003000142,Anesthesiology,TOLEDO,OH,US,M,26.000000,26.0,2.828427,...,111.000000,216.571429,327.571429,88.930000,88.930000,48.295393,54.780000,123.080000,177.860000,0
3,2012-01-01,1003000381,Physical Therapist,LADY LAKE,FL,US,M,166.800000,137.0,159.703475,...,35.000000,96.956522,274.965652,24.315331,19.488837,19.937615,8.767883,58.643913,121.576654,0
4,2012-01-01,1003000407,Family Practice,PATTON,PA,US,M,154.437500,63.5,180.288276,...,55.214521,510.000000,2343.821257,77.469027,77.795000,35.981435,29.930000,153.106164,1239.504432,0


In [56]:
a = agg_partB_df.fraudulent_mean.value_counts()
display(a)
print('Fraudulent physicians are: {0}% of all data'.format(str(np.round((a[1]/a[0])*100,decimals = 6)))) 

0    3521673
1        703
Name: fraudulent_mean, dtype: int64

Fraudulent physicians are: 0.019962% of all data


In [ ]:
# Write data for visualization before going further
%timeit agg_partB_df.to_csv('labeled-data-visualization-v1')

In [6]:
# Normalize predictors to [0,1] min-max scale
from sklearn import preprocessing

# first, get predictors (x's) scaled
df = agg_partB_df.iloc[:,7:(agg_partB_df.shape[1]-1)] 
x  = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, columns=df.columns, index=df.index)

In [7]:
# merge back in with npi and label
agg_partB_df['npi_'] = agg_partB_df['npi_'].astype(str)
agg_partB_df['year_']=agg_partB_df['year_'].astype(str).str[:4]

agg_partB_df = pd.concat([ agg_partB_df.iloc[:,[0,1,2,6,37]], df ], axis=1)

In [8]:
# One-hot encoding
for col in ['year_','nppes_provider_gender_','provider_type_']:
    agg_partB_df = pd.concat([agg_partB_df, pd.get_dummies(agg_partB_df[col])], axis=1)
    agg_partB_df = agg_partB_df.drop(col, 1) #drop old column that's been encoded

In [9]:
display(agg_partB_df.shape)
print('There are {0} predictors.'.format(agg_partB_df.shape[1]-1)) 

(3522376, 147)

There are 146 predictors.


In [ ]:
# Write data for training
%timeit agg_partB_df.to_csv('labeled-data-training-v1')